This tutorial will show you how to [install Milvus from source code](https://github.com/milvus-io/milvus/blob/master/INSTALL.md) and run the [python code](https://milvus-io.github.io/milvus-sdk-python/pythondoc/v0.2.14/index.html).

## Install Milvus from source code

### Download the code and Compile

Step 1 Download Milvus source code

In [3]:
!git clone -b 1.1 https://github.com/milvus-io/milvus.git

Cloning into 'milvus'...
remote: Enumerating objects: 92266, done.
remote: Counting objects: 100% (617/617), done.
remote: Compressing objects: 100% (375/375), done.
remote: Total 92266 (delta 273), reused 460 (delta 210), pack-reused 91649
Receiving objects: 100% (92266/92266), 72.47 MiB | 23.58 MiB/s, done.
Resolving deltas: 100% (68087/68087), done.


Step 2 Install dependencies

In [4]:
% cd /content/milvus/core
! ./ubuntu_build_deps.sh

/content/milvus/core
--2021-05-13 06:58:13--  https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB
Resolving apt.repos.intel.com (apt.repos.intel.com)... 23.218.183.111, 2600:1408:2000:690::4b23, 2600:1408:2000:686::4b23
Connecting to apt.repos.intel.com (apt.repos.intel.com)|23.218.183.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1914 (1.9K) [binary/octet-stream]
Saving to: ‘/tmp/GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB’

GPG-PUB-KEY-INTEL-S 100%[===================>]   1.87K  --.-KB/s    in 0s      

2021-05-13 06:58:14 (480 MB/s) - ‘/tmp/GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB’ saved [1914/1914]

OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://apt.repos.intel.com/mkl all InRelease [4,438 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://d

Step 3 Build Milvus source code

In [5]:
% cd /content/milvus/core  
!ls
!./build.sh -t Release
# If you want to build GPU version, add -g option, and Edit the Notebook settings with GPU
# !./build.sh -t Release -g

Streaming output truncated to the last 5000 lines.
[CXX]     Compiling src/core/ext/filters/client_channel/lb_policy/round_robin/round_robin.cc
In file included from src/core/ext/upb-generated/envoy/type/percent.upb.h:12:0,
                 from src/core/ext/upb-generated/envoy/type/percent.upb.c:11:
third_party/upb/upb/generated_util.h: In function ‘_upb_array_resize_accessor’:
third_party/upb/upb/generated_util.h:44:61: warning: unused parameter ‘type’ [-Wunused-parameter]
                                             upb_fieldtype_t type,
                                                             ^~~~
In file included from src/core/ext/upb-generated/envoy/type/range.upb.h:12:0,
                 from src/core/ext/upb-generated/envoy/type/range.upb.c:11:
third_party/upb/upb/generated_util.h: In function ‘_upb_array_resize_accessor’:
third_party/upb/upb/generated_util.h:44:61: warning: unused parameter ‘type’ [-Wunused-parameter]
                                             upb_fieldt

### Launch Milvus server

Add `lib/` directory to `LD_LIBRARY_PATH`

In [6]:
% cd /content/milvus/core/milvus
! echo $LD_LIBRARY_PATH
import os
os.environ['LD_LIBRARY_PATH'] +=":/content/milvus/core/milvus/lib"
! echo $LD_LIBRARY_PATH

/content/milvus/core/milvus
/usr/local/nvidia/lib:/usr/local/nvidia/lib64
/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/milvus/core/milvus/lib


Start Milvus server and hang

In [7]:
% cd scripts
! ls
! nohup ./start_server.sh &

/content/milvus/core/milvus/scripts
migration  start_server.sh  stop_server.sh
nohup: appending output to 'nohup.out'


Show Milvus server status

If you can find the following content, the Milvus server is successfully running: `Milvus server started successfully!`

In [8]:
! ls
! cat nohup.out

migration  myeasylog.log  nohup.out  start_server.sh  stop_server.sh

    __  _________ _   ____  ______    
   /  |/  /  _/ /| | / / / / / __/    
  / /|_/ // // /_| |/ / /_/ /\ \    
 /_/  /_/___/____/___/\____/___/     

Welcome to use Milvus!
Milvus Release version: v1.1.0, built at 2021-05-13 06:59.55, with OpenBLAS library.
You are using Milvus CPU edition
Last commit id: 2e4aaff531bd8e512b8b5035f142e3c44705c26c

Loading configuration from: ../conf/server_config.yaml
WARNNING: You are using SQLite as the meta data management, which can't be used in production. Please change it to MySQL!
Supported CPU instruction sets: avx2, sse4_2
FAISS hook AVX2
Milvus server started successfully!


## Run Python Code

### Install Milvus Python SDK

In [1]:
! pip install pymilvus==1.1.0
! pip show pymilvus

Name: pymilvus
Version: 1.1.0
Summary: Python Sdk for Milvus
Home-page: https://github.com/milvus-io/pymilvus
Author: Milvus Team
Author-email: milvus-team@zilliz.com
License: Apache-2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: grpcio, grpcio-tools, ujson, requests
Required-by: 


### Import pymilvus

In [9]:
import random
import numpy as np

from milvus import *

# Milvus server IP address and port.
# You may need to change _HOST and _PORT accordingly.
_HOST = '127.0.0.1'
_PORT = '19530'  # default value
# _PORT = '19121'  # default http value

# Vector parameters
_DIM = 8  # dimension of vector

_INDEX_FILE_SIZE = 32  # max file size of stored index

# the demo name.
collection_name = 'example_collection_'
partition_tag = 'demo_tag_'
segment_name= ''

# 10 vectors with 8 dimension, per element is float32 type, vectors should be a 2-D array
vectors = [[random.random() for _ in range(_DIM)] for _ in range(10)]
ids = [i for i in range(10)]

### Connect Milvus and check version

In [10]:
# Connect to Milvus Server 
milvus = Milvus(_HOST, _PORT)

# Close client instance
# milvus.close()

# Returns the status of the Milvus server.
server_status = milvus.server_status(timeout=10)
print(server_status)

(Status(code=0, message='Success!'), 'OK')


In [11]:
# Returns the version of the client.
client_version= milvus.client_version()
print(client_version)

# Returns the version of the Milvus server.
server_version = milvus.server_version(timeout=10)
print(server_version)

1.1.0
(Status(code=0, message='Success!'), '1.1.0')


### Check and create collection/index

In [12]:
# Checks whether a collection exists.
milvus.has_collection(collection_name=collection_name, timeout=10)

(Status(code=0, message='Success'), False)

In [13]:
# Information needed to create a collection.Defult index_file_size=1024 and metric_type=MetricType.L2
param={'collection_name':collection_name, 'dimension': _DIM, 'index_file_size': _INDEX_FILE_SIZE, 'metric_type': MetricType.L2}
# Creates a collection.
milvus.create_collection(param, timeout=10)

Status(code=0, message='Create collection successfully!')

In [14]:
# Checks whether a partition exists.
milvus.has_partition(collection_name=collection_name, partition_tag=partition_tag)

(Status(code=0, message='OK'), False)

In [15]:
# Create a partition for a collection.
milvus.create_partition(collection_name=collection_name, partition_tag=partition_tag, timeout=10)

Status(code=0, message='OK')

In [16]:
ivf_param = {'nlist': 16384}
# Creates index for a collection.
milvus.create_index(collection_name=collection_name, index_type=IndexType.IVF_FLAT, params=ivf_param)

Status(code=0, message='Build index successfully!')

### Insert data and flush

In [17]:
# Insert vectors to a collection.
milvus.insert(collection_name=collection_name, records=vectors, ids=ids)

(Status(code=0, message='Add vectors successfully!'),
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [18]:
# Flushes vector data in one collection or multiple collections to disk.
milvus.flush(collection_name_array=[collection_name], timeout=None)

Status(code=0, message='OK')

### Load data and search

In [19]:
# Loads a collection for caching.
milvus.load_collection(collection_name=collection_name, timeout=None)

Status(code=0, message='OK')

In [20]:
# Search vectors in a collection.
search_param = { "nprobe": 16 }
milvus.search(collection_name=collection_name,query_records=[vectors[0]],partition_tags=None,top_k=10,params=search_param)

(Status(code=0, message='Search vectors successfully!'), [
 [
 (id:0, distance:0.0),
 (id:9, distance:0.46119970083236694),
 (id:5, distance:0.49673816561698914),
 (id:7, distance:0.6954410672187805),
 (id:1, distance:0.8963985443115234),
 (id:8, distance:1.0356143712997437),
 (id:6, distance:1.2265537977218628),
 (id:2, distance:1.3645379543304443),
 (id:3, distance:1.4025874137878418),
 (id:4, distance:1.4224138259887695)
 ]
 ])

### Get info about collection/index

In [21]:
# Returns collection list.
milvus.list_collections(timeout=10)

(Status(code=0, message='Show collections successfully!'),
 ['example_collection_'])

In [22]:
# Show all partitions in a collection.
milvus.list_partitions(collection_name=collection_name, timeout=10)

(Status(code=0, message='Success'),
 [(collection_name='example_collection_', tag='_default'),
  (collection_name='example_collection_', tag='demo_tag_')])

In [23]:
# Returns the number of vectors in a collection.
milvus.count_entities(collection_name=collection_name, timeout=10)

(Status(code=0, message='Success!'), 10)

In [24]:
# Returns information of a collection.
milvus.get_collection_info(collection_name=collection_name, timeout=10)

(Status(code=0, message='Describe collection successfully!'),
 CollectionSchema(collection_name='example_collection_', dimension=8, index_file_size=32, metric_type=<MetricType: L2>))

In [25]:
# Returns collection statistics information
milvus.get_collection_stats(collection_name=collection_name, timeout=10)

(Status(code=0, message='Success'),
 {'partitions': [{'row_count': 10,
    'segments': [{'data_size': 400,
      'index_name': 'IDMAP',
      'name': '1620891904668048000',
      'row_count': 10}],
    'tag': '_default'},
   {'row_count': 0, 'segments': None, 'tag': 'demo_tag_'}],
  'row_count': 10})

In [26]:
# Show index information of a collection.
milvus.get_index_info(collection_name=collection_name, timeout=10)

(Status(code=0, message='Successfully'),
 IndexParam(_collection_name='example_collection_', _index_type=<IndexType: IVF_FLAT>, _params={'nlist': 16384}))

### Get data by ids

In [27]:
# List the ids in segment
# you can get the segment_name list by get_collection_stats() function.
milvus.list_id_in_segment(collection_name =collection_name, segment_name='1600328539015368000', timeout=None)

(Status(code=1, message='Segment does not exist'), [])

In [28]:
# Returns raw vectors according to ids, and you can get the ids list by list_id_in_segment() function.
# ids (list) -- list of vector id
milvus.get_entity_by_id(collection_name=collection_name, ids=[0], timeout=None)

(Status(code=0, message='Obtain vector successfully'),
 [[0.578181803226471,
   0.8928974270820618,
   0.5257136821746826,
   0.5873235464096069,
   0.6100190877914429,
   0.5913563370704651,
   0.42265674471855164,
   0.9158957600593567]])

### Get and set the configurations

In [29]:
# Gets Milvus configurations.
milvus.get_config(parent_key='cache', child_key='cache_size')

(Status(code=0, message='Success!'), '4GB')

In [30]:
# Sets Milvus configurations.
milvus.set_config(parent_key='cache', child_key='cache_size', value='5G')

(Status(code=0, message='Success!'), '')

### Delete data/index

In [31]:
# Removes an index.
milvus.drop_index(collection_name=collection_name, timeout=None)

Status(code=0, message='OK')

In [32]:
# Deletes a partition in a collection.
milvus.drop_partition(collection_name=collection_name, partition_tag=partition_tag, timeout=None)

Status(code=0, message='OK')

In [33]:
# Deletes vectors in a collection by vector ID.
# id_array (list[int]) -- list of vector id
milvus.delete_entity_by_id(collection_name=collection_name, id_array=[0], timeout=None)

Status(code=0, message='OK')

In [34]:
# Deletes a collection by name.
# milvus.drop_collection(collection_name=collection_name, timeout=10)